# Cloud masking, normalisation and mosaic production

In [ ]:
'''
## ############################## 
##
## Script Name: S2 Mosaic Creation
##
## Description: Generates Cloud/Cloud shadow free Sentinel 2 mosaics based on user provided dates.
##
## Author: Max Fancourt, Natural England
## 
## Licence: MIT Licence  
## Date Created: 2024-07-17
## 
## Date Last Modified: 2024-07-17
## 
## Versioning: v1.0
## Python Version: 3.11.5
## Dependencies:
## 
## ee       v.0.1.358
## folium   v.0.14.0
##
## ############################## ##
'''

In [1]:
# Import libraries
import ee
import folium
from folium import plugins

# Authenticate connection with Earth Engine, if exception then attempt authentication and then reinitialise connection
try:
    ee.Initialize()
except:
    # Trigger the authentication flow.
    ee.Authenticate()
    ee.Initialize()

In [2]:
# Assest import
TABLE = ee.FeatureCollection("users/MaxFancourt/all_zones_buffered")
RIVERS = ee.FeatureCollection("users/MaxFancourt/rivers2")

# Global variables
ZONE = 14  # Zone to create mosaic for
CLD_PRB_THRESH = 50 # Threshold applied to cloud probability layer to create cloud mask (any pixels with higher than this threshold will be labelled cloud)
CLD_PRJ_DIST = 3 # Distance from cloud to search for potential cloud shadow (km)
NIR_DARK_THRESH = 0.15 # NIR threshold for determining true cloud shadow from intiially determined cloud shadow area 
CLOUD_FILTER_ROUGH = 70 # Initial rough threshold based on overall image cloudiness to reduce processing overhead
START_DATE = '2023-06-01' # Start date for mosaicking
END_DATE = '2023-07-31' # End date for mosaicking
CRS = 'EPSG:27700' # Projection to export data in
EXPORT_FOLDER = "s2_mosaics"
METHOD = "dd" # "dd" or "lc" 
# "dd" = date difference mosaic, where the least cloud image is used as primary image, and the rest of the stack is then ordered in ascening absolute time difference to this image
# "lc" = lest cloud mosaic, where the image stack is ordered by cloudiness

# Spring dates 
#START_DATE = '2023-03-01'
#END_DATE = '2023-05-30'

# Autumn dates
#START_DATE = '2022-08-01'
#END_DATE = '2022-10-31'

# Summerdates
#START_DATE = '2023-06-01'
#END_DATE = '2023-07-31'

# Visulisation parameters
L2ADISPLAY = {'bands': ['B4', 'B3', 'B2'], 'min': 0, 'max': 2500, 'gamma': 1.1}

basemaps = {

    'Google Maps': folium.TileLayer(
        tiles = 'https://mt1.google.com/vt/lyrs=m&x={x}&y={y}&z={z}',
        attr = 'Google',
        name = 'Google Maps',
        overlay = True,
        control = True
    ),
    'Google Satellite': folium.TileLayer(
        tiles = 'https://mt1.google.com/vt/lyrs=s&x={x}&y={y}&z={z}',
        attr = 'Google',
        name = 'Google Satellite',
        overlay = True,
        control = True
    ),
    'Google Terrain': folium.TileLayer(
        tiles = 'https://mt1.google.com/vt/lyrs=p&x={x}&y={y}&z={z}',
        attr = 'Google',
        name = 'Google Terrain',
        overlay = True,
        control = True
    ),
    'Google Satellite Hybrid': folium.TileLayer(
        tiles = 'https://mt1.google.com/vt/lyrs=y&x={x}&y={y}&z={z}',
        attr = 'Google',
        name = 'Google Satellite',
        overlay = True,
        control = True
    ),
    'Esri Satellite': folium.TileLayer(
        tiles = 'https://server.arcgisonline.com/ArcGIS/rest/services/World_Imagery/MapServer/tile/{z}/{y}/{x}',
        attr = 'Esri',
        name = 'Esri Satellite',
        overlay = True,
        control = True
    )
}

In [3]:
# Functions 
# Get s2_sr and s2_cloudless and join on system:index 
def prepare_s2_and_cloudless(aoi, start_date, end_date):
    # Import and filter S2 surface reflectance
    s2_sr_col = ee.ImageCollection(ee.ImageCollection("COPERNICUS/S2_SR_HARMONIZED")
                                   .filterBounds(aoi.geometry())
                                   .filterDate(start_date, end_date)
                                   .filter(ee.Filter.lte('CLOUDY_PIXEL_PERCENTAGE', CLOUD_FILTER_ROUGH))
                                   .filter(ee.Filter.lte('SNOW_ICE_PERCENTAGE', 10))
                                   )

    # Import and filter the S2 cloudless data
    s2_cloudless_col = ee.ImageCollection(ee.ImageCollection("COPERNICUS/S2_CLOUD_PROBABILITY")
                                          .filterBounds(aoi.geometry())
                                          .filterDate(start_date, end_date))
    

    

    # Join the filtered s2cloudless collection to the SR collection by the 'system:index' property.
    return ee.ImageCollection(ee.Join.saveFirst('s2cloudless').apply(**{
        'primary': s2_sr_col,
        'secondary': s2_cloudless_col,
        'condition': ee.Filter.equals(**{
            'leftField': 'system:index',
            'rightField': 'system:index'
        })
    }))

# Add cloud probabilility and binary cloud mask based on CLD_PRB_THRESH to s2_sr from s2_cloudless
def add_prob_and_cloud_class(image):
  # Get the s2 cloudless and select the probability band
  cloud_prob = ee.Image(image.get('s2cloudless')).select('probability')
  # Create a binary cloud mask based on cloud threshold
  is_cloud = cloud_prob.gt(CLD_PRB_THRESH).rename("clouds")
  # Combine and return
  return ee.Image(image).addBands(ee.Image([cloud_prob, is_cloud]))

# Function to identify cloud shadows using dark pixel candidates and projection from the cloud to identify, account for water
def add_shadow_band(image):
  # Identify water areas using a combination of SCL and NDVI/NWDI thresholds
  not_water = ee.Image(image).select('SCL').neq(6)
  sat_def = ee.Image(image).select('SCL').eq(1).rename("sat_def")
  snow_ice = ee.Image(image).select('SCL').eq(11).rename("snow")
  ndwi_mask = image.normalizedDifference(['B3', 'B8']).lte(0.1)
  ndvi_mask = image.normalizedDifference(['B8', 'B4']).gte(0)
  ndvi_ndwi_mask = ndwi_mask.add(ndvi_mask).clamp(0,1).neq(0)
  riversMask = ee.Image.constant(1).clip(RIVERS).mask().Not()
  final_water_mask = not_water.add(ndvi_ndwi_mask).add(riversMask).eq(3)
  
  # Get the cirrus layer from SCL for cirrus cloud masking later 
  cirrus_mask = image.select('SCL').eq(10).eq(1).rename("cirrus_mask")
  
  # Identify dark NIR pixels that are not water (candidate cloud shadow pixels)
  dark_pixels = image.select('B8').lt(NIR_DARK_THRESH*1e4).multiply(final_water_mask).rename('dark_pixels')
  
  # Determine the direction to project cloud shadow from clouds (assumes UTM projection).
  shadow_azimuth_1 = ee.Number(image.get('MEAN_SOLAR_AZIMUTH_ANGLE_AVERAGE')).subtract(80)
  shadow_azimuth_2 = ee.Number(image.get('MEAN_SOLAR_AZIMUTH_ANGLE_AVERAGE')).subtract(90)
  shadow_azimuth_3 = ee.Number(image.get('MEAN_SOLAR_AZIMUTH_ANGLE_AVERAGE')).subtract(100)
  shadow_azimuth_4 = ee.Number(image.get('MEAN_SOLAR_AZIMUTH_ANGLE_AVERAGE')).subtract(110)
  
  # Add cirrus to the exisiting cloud layer 
  final_clouds = image.select('clouds').add(cirrus_mask).clamp(0,1)

  # Project shadows from clouds for the distance specified by CLD_PRJ_DIST 
  cloud_transform_1 = (final_clouds.directionalDistanceTransform(shadow_azimuth_1, CLD_PRJ_DIST*10)
      .reproject(**{'crs': image.select(0).projection(), 'scale': 100})
      .select('distance')
      .mask()
      .rename('cloud_transform'))
  
  cloud_transform_2 = (final_clouds.directionalDistanceTransform(shadow_azimuth_2, CLD_PRJ_DIST*10)
    .reproject(**{'crs': image.select(0).projection(), 'scale': 100})
    .select('distance')
    .mask()
    .rename('cloud_transform'))
    
  cloud_transform_3 = (final_clouds.directionalDistanceTransform(shadow_azimuth_3, CLD_PRJ_DIST*10)
    .reproject(**{'crs': image.select(0).projection(), 'scale': 100})
    .select('distance')
    .mask()
    .rename('cloud_transform'))
    
  cloud_transform_4 = (final_clouds.directionalDistanceTransform(shadow_azimuth_4, CLD_PRJ_DIST*10)
    .reproject(**{'crs': image.select(0).projection(), 'scale': 100})
    .select('distance')
    .mask()
    .rename('cloud_transform'))

  cloud_transform = cloud_transform_1.add(cloud_transform_2).add(cloud_transform_3).add(cloud_transform_4).clamp(0,1)

  # Identify the intersection of dark pixels with cloud shadow projection.
  shadows = cloud_transform.multiply(dark_pixels).rename('shadows')
  
  # Add dark pixels, cloud projection, and identified shadows and currus mask as image bands.
  return image.addBands(ee.Image([dark_pixels, cloud_transform, shadows, cirrus_mask, sat_def, snow_ice]))

# Function to assemble all cloud and cloud shadow components into a final mask
def create_final_cloud_shadow_mask(image):
   # Add cloud shadow component bands 
  cloud_shadow_image = add_shadow_band(image)
  
  # Combine cloud and shadow masks, set cloud and shadow as value 1 everything else 0
  is_cloud_shadow = cloud_shadow_image.select('clouds').add(cloud_shadow_image.select('shadows')).add(cloud_shadow_image.select('cirrus_mask')).add(cloud_shadow_image.select('sat_def')).add(cloud_shadow_image.select('snow')).gt(0)

  # Remove small cloud-shadow patches, and then dilate the remaining pixels by BUFFER input
  # 20m scale chosen to speed things up
  is_cld_shdw = (is_cloud_shadow.focalMin(2).focalMax(100*2/20)
                 .reproject(**{'crs': image.select([0]).projection(), 'scale': 20})
                 .rename('cloudmask'))
          
  # Add the final cloud-shadow mask to the image
  return cloud_shadow_image.addBands(is_cld_shdw)

# Take the joined data and join into solar days 
def create_solar_day_mosaics(joined_data):
  # Get all unique dates in this group
  tile_dates = joined_data.aggregate_array('system:time_start').distinct()
  
  # Round these to the same solar day 
  def RoundToDay(date):
    raw_date = ee.Date(date)
    return ee.Date.fromYMD(raw_date.get("year"), raw_date.get("month"), raw_date.get("day"))
  
  unique_tile_dates = tile_dates.map(RoundToDay).distinct()
  
  def JoinBySolarDay(date):
    date = ee.Date(date)
    # Filter down to images all collected on the same day
    solar_day_filtered = joined_data.filter(ee.Filter.calendarRange(date.get("month"), date.get("month"), "MONTH")).filter(ee.Filter.calendarRange(date.get("day"), date.get("day"), "DAY_OF_MONTH"))
    # Mosaic and calculate the average cloud cover across all tiles for sorting later
    return ee.Image(solar_day_filtered.mosaic()
                    .set("CLOUDY_PIXEL_PERCENTAGE_AVERAGE", solar_day_filtered.aggregate_mean("CLOUDY_PIXEL_PERCENTAGE"))
                    .set("MEAN_SOLAR_AZIMUTH_ANGLE_AVERAGE",solar_day_filtered.aggregate_mean("MEAN_SOLAR_AZIMUTH_ANGLE"))
                    .set('system:time_start', date.millis()))

  # Convert tiles into solar mosaics stack and return 
  return ee.ImageCollection(unique_tile_dates.map(JoinBySolarDay))

# Date difference 
def AddDateDifference(primary, img_collection):
  def worker(image):
    return image.set("date_difference", ee.Date(primary.get('system:time_start')).difference(image.get('system:time_start'),'hour').abs())
  return img_collection.map(worker)

# Apply cloud mask 
def apply_mask(image):
  return image.updateMask(image.select("cloudmask").eq(0))

# Define a method for displaying Earth Engine image tiles on a folium map
def add_ee_layer(self, ee_object, vis_params, name):
    try:    
        # display ee.Image()
        if isinstance(ee_object, ee.image.Image):    
            map_id_dict = ee.Image(ee_object).getMapId(vis_params)
            folium.raster_layers.TileLayer(
            tiles = map_id_dict['tile_fetcher'].url_format,
            attr = 'Google Earth Engine',
            name = name,
            overlay = True,
            control = True
            ).add_to(self)
        # display ee.ImageCollection()
        elif isinstance(ee_object, ee.imagecollection.ImageCollection):    
            ee_object_new = ee_object.mosaic()
            map_id_dict = ee.Image(ee_object_new).getMapId(vis_params)
            folium.raster_layers.TileLayer(
            tiles = map_id_dict['tile_fetcher'].url_format,
            attr = 'Google Earth Engine',
            name = name,
            overlay = True,
            control = True
            ).add_to(self)
        # display ee.Geometry()
        elif isinstance(ee_object, ee.geometry.Geometry):    
            folium.GeoJson(
            data = ee_object.getInfo(),
            name = name,
            overlay = True,
            control = True
        ).add_to(self)
        # display ee.FeatureCollection()
        elif isinstance(ee_object, ee.featurecollection.FeatureCollection):  
            ee_object_new = ee.Image().paint(ee_object, 0, 2)
            map_id_dict = ee.Image(ee_object_new).getMapId(vis_params)
            folium.raster_layers.TileLayer(
            tiles = map_id_dict['tile_fetcher'].url_format,
            attr = 'Google Earth Engine',
            name = name,
            overlay = True,
            control = True
        ).add_to(self)
    
    except:
        print("Could not display {}".format(name))
    
# Add EE drawing method to folium.
folium.Map.add_ee_layer = add_ee_layer

In [4]:
# Main script body 
# Find AOI for spatial clipping
AOI = ee.Feature(TABLE.filter(ee.Filter.eq('Zone', ZONE)).first().geometry())

# Join s2 and s2 cloudless data 
s2_joined = prepare_s2_and_cloudless(AOI, START_DATE, END_DATE)

# Identify and cloud and classify into cloud vs cloudfree
s2_cloudbands_added = s2_joined.map(add_prob_and_cloud_class)

# Create solar stacks  
solar_stacks = create_solar_day_mosaics(s2_cloudbands_added)

# Remove cloud shadow using classified cloud layer 
s2_cloud_and_cloud_shadow_masked = solar_stacks.map(create_final_cloud_shadow_mask)

# Define the primary image as the least cloudy image
primary_image_dates = s2_cloud_and_cloud_shadow_masked.sort('CLOUDY_PIXEL_PERCENTAGE_AVERAGE', True).first()

# Calculate date difference 
solar_stacks_with_date = AddDateDifference(primary_image_dates, s2_cloud_and_cloud_shadow_masked)

# Apply cloud mask to all images
solar_stacks_with_date_masked = solar_stacks_with_date.map(apply_mask)

# Calculate export mosaic based on user definied methodology
if METHOD == "lc":
    export_mosaic = solar_stacks_with_date_masked.sort('CLOUDY_PIXEL_PERCENTAGE_AVERAGE', False).mosaic().clip(AOI)
# Calculate date difference mosaic 
elif METHOD == "dd":
    export_mosaic = solar_stacks_with_date_masked.sort('date_difference', False).mosaic().clip(AOI)
else:
    print("Incorrect Method definied, needs to be either lc or dd")

# Export mosaic to attached google drive as a geotiff, note cloud/cloud shadow masked regions will be exported with value 0 and will need to be converted to NA 
task = ee.batch.Export.image.toDrive(export_mosaic.select('B.+'),
                                    description = f"zone_{ZONE}",
                                    folder = EXPORT_FOLDER,
                                    region = AOI.geometry(),
                                    scale = 10,
                                    crs = CRS,
                                    maxPixels = 1e10)
task.start()

In [14]:
# Add the created mosiacis to a folium map for manual visuationation
# Create folium base map
f = folium.Figure(width=2000, height=1000)
my_map = folium.Map(zoom_start=12).add_to(f)

# Add google baselayer
basemaps['Google Maps'].add_to(my_map)

# Add layers
my_map.add_ee_layer(AOI.geometry(), {},"AOI") # Add AOI
my_map.add_ee_layer(export_mosaic, L2ADISPLAY,"export_mosaic") # Add Mosaic

# Add a layer control panel to the map
my_map.add_child(folium.LayerControl())

display(my_map)

